In [1]:
from __future__ import division
from __future__ import print_function

import time
import numpy as np

import torch
import torch.nn.functional as F
import torch.optim as optim

from utils import load_data, accuracy, run_experiment
from models import GCN_3, ite_GCN

In [2]:
adj, features, labels, idx_train, idx_val, idx_test = load_data(path="../data/cora/", dataset="cora")

Loading cora dataset...


In [3]:
hidden = 16
dropout = 0.5
lr = 0.01
weight_decay = 5e-4
num_epochs = 200

In [4]:
model1 = GCN_3(nfeat=features.shape[1],
            nhid=hidden,
            nclass=labels.max().item() + 1,
            dropout=dropout)


Intialize a 3-layer GCN


In [5]:
model2 = ite_GCN(nfeat=features.shape[1],
            nclass=labels.max().item() + 1,
            dropout=dropout,
            nite = 3,
            allow_grad=True)

Initialize a 1-layer GCN with  3 iterations
Gradient flows to all iterations:  True


In [6]:
run_experiment(num_epochs=num_epochs, model=model1, lr=lr, weight_decay=weight_decay, features=features, adj=adj, idx_train=idx_train, idx_val=idx_val, idx_test=idx_test, labels=labels)


runrunrun!
Epoch: 0001 loss_train: 1.9593 acc_train: 0.1286 loss_val: 1.9487 acc_val: 0.1567 time: 0.0466s
Epoch: 0002 loss_train: 1.9468 acc_train: 0.2000 loss_val: 1.9249 acc_val: 0.1567 time: 0.0093s
Epoch: 0003 loss_train: 1.9269 acc_train: 0.1857 loss_val: 1.9029 acc_val: 0.1567 time: 0.0088s
Epoch: 0004 loss_train: 1.8978 acc_train: 0.2000 loss_val: 1.8827 acc_val: 0.1567 time: 0.0086s
Epoch: 0005 loss_train: 1.8773 acc_train: 0.2429 loss_val: 1.8639 acc_val: 0.1900 time: 0.0082s
Epoch: 0006 loss_train: 1.8667 acc_train: 0.2714 loss_val: 1.8465 acc_val: 0.3500 time: 0.0096s
Epoch: 0007 loss_train: 1.8544 acc_train: 0.2643 loss_val: 1.8308 acc_val: 0.3500 time: 0.0088s
Epoch: 0008 loss_train: 1.8250 acc_train: 0.2929 loss_val: 1.8168 acc_val: 0.3500 time: 0.0084s
Epoch: 0009 loss_train: 1.8142 acc_train: 0.3143 loss_val: 1.8048 acc_val: 0.3500 time: 0.0093s
Epoch: 0010 loss_train: 1.8091 acc_train: 0.2929 loss_val: 1.7951 acc_val: 0.3500 time: 0.0112s
Epoch: 0011 loss_train: 1.801

In [7]:
run_experiment(num_epochs=num_epochs, model=model2, lr=lr, weight_decay=weight_decay, features=features, adj=adj, idx_train=idx_train, idx_val=idx_val, idx_test=idx_test, labels=labels)


runrunrun!
Epoch: 0001 loss_train: 1.9433 acc_train: 0.1857 loss_val: 2.3318 acc_val: 0.3500 time: 0.6795s
Epoch: 0002 loss_train: 2.4178 acc_train: 0.2929 loss_val: 1.9849 acc_val: 0.1267 time: 0.7625s
Epoch: 0003 loss_train: 1.9770 acc_train: 0.1357 loss_val: 1.9384 acc_val: 0.1567 time: 0.6224s
Epoch: 0004 loss_train: 1.9376 acc_train: 0.2214 loss_val: 1.9257 acc_val: 0.3467 time: 0.6431s
Epoch: 0005 loss_train: 1.9244 acc_train: 0.3071 loss_val: 1.8787 acc_val: 0.3500 time: 0.6035s
Epoch: 0006 loss_train: 1.8806 acc_train: 0.2786 loss_val: 1.8154 acc_val: 0.3500 time: 0.6098s
Epoch: 0007 loss_train: 1.8079 acc_train: 0.2786 loss_val: 1.8537 acc_val: 0.3500 time: 0.6281s
Epoch: 0008 loss_train: 1.8752 acc_train: 0.2929 loss_val: 1.7720 acc_val: 0.3500 time: 0.6449s
Epoch: 0009 loss_train: 1.7858 acc_train: 0.2929 loss_val: 1.7980 acc_val: 0.3500 time: 0.6176s
Epoch: 0010 loss_train: 1.8074 acc_train: 0.2929 loss_val: 1.8009 acc_val: 0.3500 time: 0.6209s
Epoch: 0011 loss_train: 1.808

In [8]:
model3 = ite_GCN(nfeat=features.shape[1],
            nclass=labels.max().item() + 1,
            dropout=dropout,
            nite = 3,
            allow_grad=False)

Initialize a 1-layer GCN with  3 iterations
Gradient flows to all iterations:  False


In [9]:
run_experiment(num_epochs=num_epochs, model=model3, lr=lr, weight_decay=weight_decay, features=features, adj=adj, idx_train=idx_train, idx_val=idx_val, idx_test=idx_test, labels=labels)


runrunrun!
Epoch: 0001 loss_train: 1.9459 acc_train: 0.1786 loss_val: 2.2295 acc_val: 0.3500 time: 0.7255s
Epoch: 0002 loss_train: 2.2785 acc_train: 0.2929 loss_val: 1.9810 acc_val: 0.1267 time: 0.6458s
Epoch: 0003 loss_train: 1.9734 acc_train: 0.1214 loss_val: 1.9417 acc_val: 0.1567 time: 0.6761s
Epoch: 0004 loss_train: 1.9394 acc_train: 0.2000 loss_val: 1.9222 acc_val: 0.1567 time: 0.6179s
Epoch: 0005 loss_train: 1.9208 acc_train: 0.2000 loss_val: 1.8523 acc_val: 0.3733 time: 0.6073s
Epoch: 0006 loss_train: 1.8518 acc_train: 0.2071 loss_val: 1.8677 acc_val: 0.3500 time: 0.6052s
Epoch: 0007 loss_train: 1.8882 acc_train: 0.2929 loss_val: 1.7835 acc_val: 0.3500 time: 0.6147s
Epoch: 0008 loss_train: 1.7799 acc_train: 0.2929 loss_val: 1.7863 acc_val: 0.3500 time: 0.6192s
Epoch: 0009 loss_train: 1.7912 acc_train: 0.2929 loss_val: 1.7620 acc_val: 0.3500 time: 0.6445s
Epoch: 0010 loss_train: 1.7663 acc_train: 0.2929 loss_val: 1.7009 acc_val: 0.3500 time: 0.6714s
Epoch: 0011 loss_train: 1.717